In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/tmp/nerlnet/data/NerlnetData-master/nerlnet/EEG_eye_state_classification_Ohad/EEG_Eye_State_Classification.csv")

In [3]:
print(df)

       4329.23  4009.23  4289.23  4148.21  4350.26  4586.15  4096.92  4641.03  \
0      4324.62  4004.62  4293.85  4148.72  4342.05  4586.67  4097.44  4638.97   
1      4327.69  4006.67  4295.38  4156.41  4336.92  4583.59  4096.92  4630.26   
2      4328.72  4011.79  4296.41  4155.90  4343.59  4582.56  4097.44  4630.77   
3      4326.15  4011.79  4292.31  4151.28  4347.69  4586.67  4095.90  4627.69   
4      4321.03  4004.62  4284.10  4153.33  4345.64  4587.18  4093.33  4616.92   
...        ...      ...      ...      ...      ...      ...      ...      ...   
14974  4281.03  3990.26  4245.64  4116.92  4333.85  4614.36  4074.87  4625.64   
14975  4276.92  3991.79  4245.13  4110.77  4332.82  4615.38  4073.33  4621.54   
14976  4277.44  3990.77  4246.67  4113.85  4333.33  4615.38  4072.82  4623.59   
14977  4284.62  3991.79  4251.28  4122.05  4334.36  4616.41  4080.51  4628.72   
14978  4287.69  3997.44  4260.00  4121.03  4333.33  4616.41  4088.72  4638.46   

       4222.05  4238.46  42

In [4]:
columns_to_select = list(range(0, 8)) + list(range(14, 16))   #take the first 8 channels and the classes 
data = df.iloc[:, columns_to_select]
print(data)

       4329.23  4009.23  4289.23  4148.21  4350.26  4586.15  4096.92  4641.03  \
0      4324.62  4004.62  4293.85  4148.72  4342.05  4586.67  4097.44  4638.97   
1      4327.69  4006.67  4295.38  4156.41  4336.92  4583.59  4096.92  4630.26   
2      4328.72  4011.79  4296.41  4155.90  4343.59  4582.56  4097.44  4630.77   
3      4326.15  4011.79  4292.31  4151.28  4347.69  4586.67  4095.90  4627.69   
4      4321.03  4004.62  4284.10  4153.33  4345.64  4587.18  4093.33  4616.92   
...        ...      ...      ...      ...      ...      ...      ...      ...   
14974  4281.03  3990.26  4245.64  4116.92  4333.85  4614.36  4074.87  4625.64   
14975  4276.92  3991.79  4245.13  4110.77  4332.82  4615.38  4073.33  4621.54   
14976  4277.44  3990.77  4246.67  4113.85  4333.33  4615.38  4072.82  4623.59   
14977  4284.62  3991.79  4251.28  4122.05  4334.36  4616.41  4080.51  4628.72   
14978  4287.69  3997.44  4260.00  4121.03  4333.33  4616.41  4088.72  4638.46   

       0  1  
0      0  1  

In [5]:
# Define the number of samples and duration
num_samples = 14980
total_duration = 117  # seconds
sample_rate = int(num_samples / total_duration)  # Samples per second (Hz)
first_feature = data.iloc[:,:1]
print(first_feature)

       4329.23
0      4324.62
1      4327.69
2      4328.72
3      4326.15
4      4321.03
...        ...
14974  4281.03
14975  4276.92
14976  4277.44
14977  4284.62
14978  4287.69

[14979 rows x 1 columns]


In [6]:
#building the eye state of 128 combined samples
#The 8 column indicate that the eye close
#If sum == 0  --> class of batch is "0" (eye-open)
#If sum == 128  --> class of batch is "1" (eye-closed)
#if sum < 128  --> class of batch is "2" (blinking eye)

list = []
for i in range(1,total_duration):
    sum_of_batch =data.iloc[sample_rate * (i - 1):sample_rate * i, 8].sum()  # sum of window size of 128 "time samples"  of classes
    list.append(sum_of_batch)
sum = pd.DataFrame(list)
sum_data = {'value' :list}
sum_data_df = pd.DataFrame(sum_data)

new_rows = []
for index, row in sum_data_df.iterrows():
    value = row['value']
    if value == 0:
        new_row = [0, 1, 0]
    elif value == 128:
        new_row = [1, 0, 0]
    else:
        new_row = [0, 0, 1]
    new_rows.append(new_row)
class_df = pd.DataFrame(new_rows)
print(class_df)

     0  1  2
0    0  1  0
1    0  0  1
2    1  0  0
3    1  0  0
4    1  0  0
..  .. .. ..
111  0  0  1
112  0  1  0
113  0  1  0
114  0  1  0
115  0  1  0

[116 rows x 3 columns]


In [7]:

channels = 8
new_table = pd.DataFrame()
for col in range (0,channels,2):
    channel2_table = pd.DataFrame()
    count = 0
    # Define a batch:
    # Miss the last second
    for i in range(1,total_duration):
        signal = data.iloc[sample_rate * (i - 1):sample_rate * i, col:col + 2]  # window size of 128 "time samples"  from 2 channels 
        # new_sample = 128 time samples from 1 channel
        # batch size = 2 new_samples  = signal
        signal = signal.T
        #print(signal)
        # Perform FFT and get the magnitude
        fft_output = np.fft.fft(signal)
        #print(fft_output)
        fft_magnitude = np.abs(fft_output)
        #print(fft_magnitude)
        dfbatch = pd.DataFrame(fft_magnitude)
        channel2_table = pd.concat([channel2_table, dfbatch], ignore_index=True)
        count = count + 1
        #print(dfbatch)
    #print(count)
    #print(f" table of 2 channels:{channel2_table}")
    #print(channel2_table)
    new_table = pd.concat([new_table, channel2_table], ignore_index=True)
new_table  # should be 128 coulmns , rows = 116 sec * 2 channels * 4 times = 928 
# Every source will get 116 * 2 = 232 samples 

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,551653.39,235.460873,187.185488,301.879646,135.134749,113.295301,174.604972,53.620996,412.084665,56.694873,...,288.021268,56.694873,412.084665,53.620996,174.604972,113.295301,135.134749,301.879646,187.185488,235.460873
1,512834.37,197.725751,107.988455,111.231016,169.352234,61.849598,60.891852,56.644761,183.207117,154.073615,...,227.600813,154.073615,183.207117,56.644761,60.891852,61.849598,169.352234,111.231016,107.988455,197.725751
2,558953.38,5961.966483,2096.077984,876.173483,769.758560,539.488013,161.341402,291.068538,120.721822,230.855347,...,104.062276,230.855347,120.721822,291.068538,161.341402,539.488013,769.758560,876.173483,2096.077984,5961.966483
3,511681.51,6381.264203,2064.610661,906.721579,793.086052,808.486778,395.290237,77.561894,173.777461,301.126745,...,166.118217,301.126745,173.777461,77.561894,395.290237,808.486778,793.086052,906.721579,2064.610661,6381.264203
4,554213.31,1473.295018,1639.037219,476.914228,422.036651,71.584766,452.204591,117.611228,139.228332,97.050714,...,118.980118,97.050714,139.228332,117.611228,452.204591,71.584766,422.036651,476.914228,1639.037219,1473.295018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,589361.07,89.430058,168.414519,208.086264,104.051333,169.072271,168.106772,185.040579,93.918441,161.427435,...,309.909337,161.427435,93.918441,185.040579,168.106772,169.072271,104.051333,208.086264,168.414519,89.430058
924,520531.37,361.602247,106.411875,266.542667,38.439607,22.071709,91.558265,45.388413,48.536647,138.907925,...,76.405272,138.907925,48.536647,45.388413,91.558265,22.071709,38.439607,266.542667,106.411875,361.602247
925,589988.75,358.775437,299.507752,208.150477,37.346656,107.917302,58.582121,113.573190,127.560915,82.953801,...,12.589425,82.953801,127.560915,113.573190,58.582121,107.917302,37.346656,208.150477,299.507752,358.775437
926,520394.35,112.473173,48.070251,47.503189,133.194574,123.624653,66.870118,66.231807,38.268113,70.660002,...,184.372755,70.660002,38.268113,66.231807,66.870118,123.624653,133.194574,47.503189,48.070251,112.473173


In [8]:
# Parameters
k = 232
j = 0    #row index of new_tablw that should update each Iteration

#expand new_table with 3 additional columns for classes
for col in range(3):
    new_table[col + 128] = np.nan  # Start adding columns from column index 128

# Iterating over class_df
for i in range(116):
    # Rows in new_table to be updated
    if i == 0:
        row_indices = [j, j+1, k*(j+1), k*(j+1)+1, 2*(k+j), 2*(k*(j+1))+1, 3*(k*(j+1)), 3*(k*(j+1))+1]    #class of the first batch is match to rows: [0,1,232,233,464,465,696,697]
    else:  
        row_indices = [j, j+1, k+j, (k+j)+1, (2*k)+j, (2*k)+j+1, (3*k)+j, (3*k)+j+1]                      #class of the second batch is match to rows: [2,3,234,235,466,467,698,699].....

    # Updating the rows in new_table
    for row_index in row_indices:
        if row_index < len(new_table):
            new_table.iloc[row_index, -3:] = class_df.iloc[i]

    # Incrementing indices
    j += 2

new_table

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,551653.39,235.460873,187.185488,301.879646,135.134749,113.295301,174.604972,53.620996,412.084665,56.694873,...,53.620996,174.604972,113.295301,135.134749,301.879646,187.185488,235.460873,0.0,1.0,0.0
1,512834.37,197.725751,107.988455,111.231016,169.352234,61.849598,60.891852,56.644761,183.207117,154.073615,...,56.644761,60.891852,61.849598,169.352234,111.231016,107.988455,197.725751,0.0,1.0,0.0
2,558953.38,5961.966483,2096.077984,876.173483,769.758560,539.488013,161.341402,291.068538,120.721822,230.855347,...,291.068538,161.341402,539.488013,769.758560,876.173483,2096.077984,5961.966483,0.0,0.0,1.0
3,511681.51,6381.264203,2064.610661,906.721579,793.086052,808.486778,395.290237,77.561894,173.777461,301.126745,...,77.561894,395.290237,808.486778,793.086052,906.721579,2064.610661,6381.264203,0.0,0.0,1.0
4,554213.31,1473.295018,1639.037219,476.914228,422.036651,71.584766,452.204591,117.611228,139.228332,97.050714,...,117.611228,452.204591,71.584766,422.036651,476.914228,1639.037219,1473.295018,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,589361.07,89.430058,168.414519,208.086264,104.051333,169.072271,168.106772,185.040579,93.918441,161.427435,...,185.040579,168.106772,169.072271,104.051333,208.086264,168.414519,89.430058,0.0,1.0,0.0
924,520531.37,361.602247,106.411875,266.542667,38.439607,22.071709,91.558265,45.388413,48.536647,138.907925,...,45.388413,91.558265,22.071709,38.439607,266.542667,106.411875,361.602247,0.0,1.0,0.0
925,589988.75,358.775437,299.507752,208.150477,37.346656,107.917302,58.582121,113.573190,127.560915,82.953801,...,113.573190,58.582121,107.917302,37.346656,208.150477,299.507752,358.775437,0.0,1.0,0.0
926,520394.35,112.473173,48.070251,47.503189,133.194574,123.624653,66.870118,66.231807,38.268113,70.660002,...,66.231807,66.870118,123.624653,133.194574,47.503189,48.070251,112.473173,0.0,1.0,0.0


In [9]:
new_table.to_csv('EEG_Eye_State_Classification_table_freq_window.csv', index=False, header=False)